In [11]:
pip install imbalanced-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 257.7/257.7 kB 3.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.0
    Uninstalling joblib-1.1.0:
      Successfully uninstalled joblib-1.1.0
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
from datetime import datetime
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from collections import defaultdict
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import LabelEncoder


train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

extra_params = {"feature_tools" : 1, "undersampling" : 0, "oversampling" : 1, "drop_cat_vars" : 0, "date_column" : 1, "corr_thresh" : 0.9}

### Division of Sets

In [13]:
X = train.drop(columns = "target")
y = train["target"]

In [14]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [15]:
X_train['target'] = y_train
X_val['target'] = y_val

In [16]:
train = X_train.copy()
val = X_val.copy()

In [17]:
#Keep average cost min
averagecostmin = val['average cost min']
idtest = test["id"]

### Initial Exploration

In [18]:
train.describe()

,Age,L_O_S,Dropped_Calls,Peak_calls_Sum,Peak_mins_Sum,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,...,call_cost_per_min,actual call cost,Total_call_cost,Total_Cost,average cost min,Peak ratio,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,target
count,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,...,4032.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000,4035.000000
mean,31.414126,33.699347,2.673358,238.945229,708.792184,104.036431,311.938372,16.322677,50.079971,169.551549,...,10.081397,19.133955,73.501256,186.803427,0.168306,0.609716,0.327824,0.061717,0.162328,0.146716
std,12.775443,14.007828,3.418703,239.288923,503.929355,97.260823,199.167646,16.296733,36.092455,141.787096,...,2.108023,27.987193,60.000356,82.079489,0.077812,0.224430,0.209932,0.063644,0.105643,0.353867
min,12.000000,9.633333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,59.940000,0.048998,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,21.300000,0.000000,59.000000,307.500000,28.000000,149.550000,4.000000,22.800000,64.261044,...,8.573250,0.000000,32.134081,123.802373,0.134413,0.466317,0.165139,0.020942,0.078918,0.000000
50%,29.000000,33.666667,1.000000,161.000000,614.400001,73.000000,291.000000,11.000000,44.399999,132.279506,...,9.518597,7.515854,57.801016,173.254831,0.154525,0.654877,0.287722,0.043932,0.157734,0.000000
75%,39.000000,45.933333,2.000000,343.000000,1014.300000,156.000000,446.400000,24.000000,71.750000,236.853610,...,11.696253,28.308661,99.217719,232.116107,0.179302,0.782261,0.458395,0.080924,0.244154,0.000000
max,80.000000,58.200000,15.000000,1626.000000,2901.600001,560.000000,1091.099999,106.000000,205.000000,935.947864,...,21.734694,184.892166,437.063835,587.063835,1.357564,1.000000,1.000000,0.731884,2.601223,1.000000


In [19]:
train.describe(include = 'object')

,Gender,Connect_Date,tariff,Handset,Usage_Band,Tariff_OK,high Dropped calls,No Usage,id
count,4035,4035,4035,4035,4032,4035,4035,4035,4035
unique,2,1310,5,11,5,4,2,2,4035
top,F,11/07/99,CAT 200,S50,Med,OK,F,F,K277140
freq,2042,11,1802,944,2232,4006,3933,4032,1


Drop ID (too many unique values), No Usage (Only has False Values)

In [20]:
train = train.drop(columns = ["id", "No Usage"])
val = val.drop(columns = ["id", "No Usage"])
test = test.drop(columns = ["id", "No Usage"])

### Preprocessing

#### Feature Engineering

In [21]:
if extra_params["date_column"] == 1:
    train["Connect_Date"] = train["Connect_Date"].apply(lambda x : datetime.strptime(x, "%d/%m/%y"))
    test["Connect_Date"] = test["Connect_Date"].apply(lambda x : datetime.strptime(x, "%d/%m/%y"))
    val["Connect_Date"] = val["Connect_Date"].apply(lambda x : datetime.strptime(x, "%d/%m/%y"))
    
    max_train = train["Connect_Date"].max()
    max_test = test["Connect_Date"].max()
    max_val = val["Connect_Date"].max()

    train["End_Date"] = max_train
    test["End_Date"] = max_test
    val["End_Date"] = max_val

    train["Connect_Time"] = train["End_Date"] - train["Connect_Date"]
    test["Connect_Time"] = test["End_Date"] - test["Connect_Date"]
    val["Connect_Time"] = val["End_Date"] - val["Connect_Date"]

    train["Connect_Time"] = train["Connect_Time"].apply(lambda x : x.days)
    test["Connect_Time"] = test["Connect_Time"].apply(lambda x : x.days)
    val["Connect_Time"] = val["Connect_Time"].apply(lambda x : x.days)

    train = train.drop(columns = ["End_Date"])
    test = test.drop(columns = ["End_Date"])
    val = val.drop(columns = ["End_Date"])

train = train.drop(columns = ["Connect_Date"])
test = test.drop(columns = ["Connect_Date"])
val = val.drop(columns = ["Connect_Date"])

#### Missing Values

In [22]:
train.isna().sum()

Gender                    0
Age                       0
L_O_S                     0
Dropped_Calls             0
tariff                    0
Handset                   0
Peak_calls_Sum            0
Peak_mins_Sum             0
OffPeak_calls_Sum         0
OffPeak_mins_Sum          0
Weekend_calls_Sum         0
Weekend_mins_Sum          0
International_mins_Sum    0
Nat_call_cost_Sum         0
AvePeak                   0
AveOffPeak                0
AveWeekend                0
National_calls            0
National mins             0
AveNational               0
All_calls_mins            0
Dropped_calls_ratio       3
Usage_Band                3
Mins_charge               0
call_cost_per_min         3
actual call cost          0
Total_call_cost           0
Total_Cost                0
Tariff_OK                 0
average cost min          0
Peak ratio                0
OffPeak ratio             0
Weekend ratio             0
Nat-InterNat Ratio        0
high Dropped calls        0
target              

In [23]:
rows_with_missing_values = train[train.isnull().any(axis=1)]

In [24]:
rows_with_missing_values[["call_cost_per_min", "Dropped_calls_ratio", "Usage_Band"]]

,call_cost_per_min,Dropped_calls_ratio,Usage_Band
3836,NaN,NaN,NaN
4301,NaN,NaN,NaN
3237,NaN,NaN,NaN


Missing values are all in the same 3 rows. Since they're only 3 observations out of 4035, they're imputed using median imputation

##### Imputing Numerical Columns

In [25]:
# Assuming 'data' is your dataset with missing values
# Create an instance of SimpleImputer with strategy='median'
imputer = SimpleImputer(strategy='median')

# Fit the imputer on the data
imputer.fit(train[["call_cost_per_min", "Dropped_calls_ratio"]])

# Transform the data by replacing missing values with the median
test_clean = imputer.transform(test[["call_cost_per_min", "Dropped_calls_ratio"]])
train_clean = imputer.transform(train[["call_cost_per_min", "Dropped_calls_ratio"]])
val_clean = imputer.transform(val[["call_cost_per_min", "Dropped_calls_ratio"]])

In [26]:
train["call_cost_per_min"] = train_clean[:, 0]
test["call_cost_per_min"] = test_clean[:, 0]
val["call_cost_per_min"] = val_clean[:, 0]

train["Dropped_calls_ratio"] = train_clean[:, 1]
test["Dropped_calls_ratio"] = test_clean[:, 1]
val["Dropped_calls_ratio"] = val_clean[:, 1]

##### Imputing Categorical Column

In [27]:
# Initialize the imputer with 'most_frequent' strategy
imputer = SimpleImputer(strategy='most_frequent')

imputer.fit(np.array(train["Usage_Band"]).reshape(-1, 1))

train["Usage_Band"] = imputer.transform(np.array(train["Usage_Band"]).reshape(-1, 1))[:, 0]
test["Usage_Band"] = imputer.transform(np.array(test["Usage_Band"]).reshape(-1, 1))[:, 0]
val["Usage_Band"] = imputer.transform(np.array(val["Usage_Band"]).reshape(-1, 1))[:, 0]

#### Variable Selection

##### Dropping numerical variables that are useless

In [28]:
correlation_matrix = train[train.select_dtypes(include=['number']).columns.tolist()].corr()

In [29]:
corrtarget = correlation_matrix['target']

In [30]:
# Make plot for this

In [31]:
high_correlation_pairs = []
for i in range(len(correlation_matrix.columns)):
    for j in range(i+1, len(correlation_matrix.columns)):
        if abs(correlation_matrix.iloc[i, j]) > extra_params["corr_thresh"]:
            high_correlation_pairs.append((correlation_matrix.columns[i], correlation_matrix.columns[j]))


In [32]:
high_correlation_pairs

[('L_O_S', 'Connect_Time'),
 ('Peak_calls_Sum', 'National_calls'),
 ('Peak_mins_Sum', 'National mins'),
 ('Peak_mins_Sum', 'All_calls_mins'),
 ('International_mins_Sum', 'Total_call_cost'),
 ('Nat_call_cost_Sum', 'actual call cost'),
 ('National mins', 'All_calls_mins'),
 ('All_calls_mins', 'Total_Cost'),
 ('Total_call_cost', 'Total_Cost'),
 ('Peak ratio', 'OffPeak ratio')]

In [33]:
def build_adjacency_list(pairs):
    adjacency_list = defaultdict(list)
    for u, v in pairs:
        adjacency_list[u].append(v)
        adjacency_list[v].append(u)
    return adjacency_list

# Function to perform depth-first search (DFS) traversal to find connected components
def dfs(node, adjacency_list, visited, component):
    visited.add(node)
    component.append(node)
    for neighbor in adjacency_list[node]:
        if neighbor not in visited:
            dfs(neighbor, adjacency_list, visited, component)

# Function to find connected components in the graph
def find_connected_components(pairs):
    adjacency_list = build_adjacency_list(pairs)
    visited = set()
    connected_components = []
    for node in adjacency_list:
        if node not in visited:
            component = []
            dfs(node, adjacency_list, visited, component)
            connected_components.append(component)
    return connected_components

# Find connected groups
connected_groups = find_connected_components(high_correlation_pairs)

In [34]:
def get_variables_to_drop(group, corrtarget):
    max_cor = 0
    var_max = ''
    for var in group:
        if abs(corrtarget[var]) > max_cor:
            var_max = var
    group.remove(var_max)

    return group

In [35]:
variables_to_drop = [get_variables_to_drop(group, corrtarget) for group in connected_groups]

In [36]:
drop_vars = [item for sublist in variables_to_drop for item in sublist]

In [37]:
train = train.drop(columns = drop_vars)

In [38]:
val = val.drop(columns = drop_vars)

In [39]:
test = test.drop(columns = drop_vars)

##### Deleting categorical variables that are useless

In [40]:
for variable in list(train.describe(include = 'object').columns):
    print(train.groupby('target')[variable].value_counts())

target  Gender
0       F         1761
        M         1682
1       M          311
        F          281
Name: Gender, dtype: int64
target  tariff  
0       CAT 200     1533
        CAT 100      736
        Play 100     503
        Play 300     471
        CAT 50       200
1       CAT 200      269
        Play 100     128
        CAT 100      110
        Play 300      47
        CAT 50        38
Name: tariff, dtype: int64
target  Handset
0       S50        825
        BS110      599
        S80        574
        WC95       526
        ASAD170    514
        BS210      220
        CAS60       90
        ASAD90      42
        CAS30       37
        SOP20       10
        SOP10        6
1       ASAD90     174
        S50        119
        CAS30      108
        BS110       95
        SOP10       28
        SOP20       24
        S80         20
        ASAD170     10
        BS210        8
        WC95         6
Name: Handset, dtype: int64
target  Usage_Band
0       Med           1974

In [41]:
#Make plots for this

UsageBand, Tariff seem useless
Most useful inferences: High Dropped Calls seems a good indicator for whether someone will churn or not, ratio of Trues for churners is much higher. Tariff_Ok's High Cat 100 for churners is much higher than for non churners, would be good to add numerical encoding. Same for handset, numerical encoding would be nice. 

In [42]:
if extra_params["drop_cat_vars"] == 1:
    train = train.drop(columns = ["No Usage", "Usage_Band", "tariff"])
    test = test.drop(columns = ["No Usage", "Usage_Band", "tariff"])
    val = val.drop(columns = ["No Usage", "Usage_Band", "tariff"])

Outliers are not dealth with (they are useful in our detection, the higher the costs the bigger the probability of churning?)

#### Variable Transformation

##### Numerical Variables

In [43]:
test['target'] = 0

In [44]:
test

,Gender,Age,Dropped_Calls,tariff,Handset,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,...,call_cost_per_min,actual call cost,Tariff_OK,average cost min,OffPeak ratio,Weekend ratio,Nat-InterNat Ratio,high Dropped calls,Connect_Time,target
0,F,20.0,3.0,CAT 200,WC95,110.0,189.982227,8.0,5.017969,25.872662,...,8.933287,9.409262,OK,0.116924,0.173365,0.009823,0.020349,F,1314,0
1,F,54.0,0.0,Play 300,BS110,93.0,551.817423,8.0,17.593417,138.267040,...,11.931697,2.739423,OK,0.120457,0.532898,0.022391,0.124480,F,667,0
2,F,28.0,0.0,CAT 200,ASAD170,90.0,152.731006,10.0,23.907633,492.202970,...,9.560903,95.406158,OK,0.149435,0.034434,0.006660,0.225763,F,443,0
3,F,26.0,5.0,CAT 200,WC95,126.0,358.760706,0.0,3.024655,174.998124,...,8.398145,14.464628,OK,0.137372,0.263256,0.005870,0.108872,F,503,0
4,F,34.0,0.0,CAT 100,WC95,73.0,141.180280,11.0,47.034972,80.927142,...,12.766109,34.757126,OK,0.180704,0.166995,0.056390,0.090896,F,426,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,F,32.0,0.0,CAT 200,BS110,8.0,42.952890,45.0,105.774031,86.262452,...,9.588439,35.396282,OK,0.122342,0.012611,0.070297,0.035339,F,921,0
1678,M,19.0,0.0,CAT 100,BS110,-1.0,73.643988,2.0,28.751955,56.083237,...,13.794174,30.518649,OK,0.163502,0.052157,0.041489,0.079988,F,366,0
1679,F,34.0,0.0,CAT 200,S50,21.0,278.414345,2.0,34.029118,188.224017,...,9.031387,96.373049,OK,0.127485,0.106429,0.011459,0.098248,F,1373,0
1680,M,21.0,2.0,CAT 100,BS110,76.0,196.742943,-1.0,18.472483,137.376495,...,11.898371,-0.223523,OK,0.205830,0.358159,0.017720,0.256554,F,1045,0


In [45]:
# List of numerical variable names
numerical_features = train.select_dtypes(include=['number']).columns.tolist()

# List of categorical variable names (replace with actual categorical variable names)
categorical_features = train.select_dtypes(exclude=['number']).columns.tolist()

# Define the transformers
transformers = [
    ('num', MinMaxScaler(), numerical_features)
]

# Create the column transformer
preprocessor = ColumnTransformer(transformers, remainder='passthrough')

# Apply the column transformer to the data
transformed_train = preprocessor.fit_transform(train)
transformed_val = preprocessor.transform(val)
transformed_test = preprocessor.transform(test)


# Convert the transformed data back to DataFrame
transformed_train = pd.DataFrame(transformed_train, columns=numerical_features + categorical_features)
transformed_test = pd.DataFrame(transformed_test, columns=numerical_features + categorical_features)
transformed_val = pd.DataFrame(transformed_val, columns=numerical_features + categorical_features)



# Only the numerical variables will be scaled to the range [0, 1]

MinMaxScaling (0, 1)

MinMaxScaling (-1, 1)

Normalization + MinMaxScaling

##### Categorical Variables

One-Hot Encoding

In [46]:
for variable in list(train.describe(include = 'object').columns):
    print(train.groupby('target')[variable].value_counts())

target  Gender
0       F         1761
        M         1682
1       M          311
        F          281
Name: Gender, dtype: int64
target  tariff  
0       CAT 200     1533
        CAT 100      736
        Play 100     503
        Play 300     471
        CAT 50       200
1       CAT 200      269
        Play 100     128
        CAT 100      110
        Play 300      47
        CAT 50        38
Name: tariff, dtype: int64
target  Handset
0       S50        825
        BS110      599
        S80        574
        WC95       526
        ASAD170    514
        BS210      220
        CAS60       90
        ASAD90      42
        CAS30       37
        SOP20       10
        SOP10        6
1       ASAD90     174
        S50        119
        CAS30      108
        BS110       95
        SOP10       28
        SOP20       24
        S80         20
        ASAD170     10
        BS210        8
        WC95         6
Name: Handset, dtype: int64
target  Usage_Band
0       Med           1974

In [47]:
# Encode binary variables using map
binary_mapping1 = {'M': 1, 'F': 0}
binary_mapping2 = {'T': 1, 'F': 0}

transformed_train['high Dropped calls'] = transformed_train['high Dropped calls'].map(binary_mapping2)
transformed_train['Gender'] = transformed_train['Gender'].map(binary_mapping1)
transformed_test['high Dropped calls'] = transformed_test['high Dropped calls'].map(binary_mapping2)
transformed_test['Gender'] = transformed_test['Gender'].map(binary_mapping1)
transformed_val['high Dropped calls'] = transformed_val['high Dropped calls'].map(binary_mapping2)
transformed_val['Gender'] = transformed_val['Gender'].map(binary_mapping1)


# Filter and encode categorical variables
# For categorical_1, keep only 'High CAT 100', encode the rest as 'Other'
transformed_train['Tariff_OK'] = transformed_train['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)
transformed_test['Tariff_OK'] = transformed_test['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)
transformed_val['Tariff_OK'] = transformed_val['Tariff_OK'].apply(lambda x: 1 if x == 'High CAT 100' else 0)

# Encode categorical_2 using one-hot encoding
encoded_train = pd.get_dummies(transformed_train, columns=['Handset', "tariff", "Usage_Band"], dtype = int)
encoded_test = pd.get_dummies(transformed_test, columns=['Handset', "tariff", "Usage_Band"], dtype = int)
encoded_val = pd.get_dummies(transformed_val, columns=['Handset', "tariff", "Usage_Band"], dtype = int)

In [48]:
encoded_val['target'] = encoded_val['target'].astype('category')

In [49]:
X_train = encoded_train.drop(columns = "target")
y_train = encoded_train["target"]

In [50]:
X_val = encoded_val.drop(columns = "target")
y_val = encoded_val["target"]

In [51]:
X_test = encoded_test.drop(columns = "target")

In [52]:
# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the categorical labels
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)

TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

#### Oversampling / Undersampling

##### Oversampling

In [53]:
if extra_params.get("oversampling") == 1:
    if X_train is not None and y_train is not None:
        # Ensure y_train is an array of integers
        print("y_train data type:", type(y_train))

        # Apply SMOTE for oversampling
        smote = SMOTE(random_state=42)
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        # Update X_train and y_train with resampled data
        X_train = X_resampled
        y_train = y_resampled
    else:
        print("X_train or y_train is None. Please check the input data.")
else:
    print("Oversampling is disabled.")

y_train data type: <class 'numpy.ndarray'>


##### Undersampling

In [54]:
# Apply RandomUnderSampler for undersampling
if extra_params["undersampling"] == 1:
    rus = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = rus.fit_resample(X, y)

    X_train = X_resampled
    y_train = y_resampled

### Creation of Models and Results

In [55]:
clf = RandomForestClassifier(max_depth=20, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, random_state=0)

#### Validation Metrics

In [56]:
#Calculate AUC Score for Validation
y_pred = clf.predict(X_val)
fpr, tpr, thresholds = metrics.roc_curve(y_val, y_pred)
metrics.auc(fpr, tpr)

0.8553730682304074

In [57]:
def Profit_top_k(df_results, k = 20, var = 'average cost min'):
    sorted_df = df_results.sort_values(by= "predict_proba", ascending = False)
    sum = 0
    for i in range(0, k):
        if df_results['y_true'][i] == df_results['y_pred'][i]:
            sum += df_results[var][i]
    return sum

In [58]:
y_pred_proba = clf.predict_proba(X_val)
y_pred_proba = [sublist[1] for sublist in y_pred_proba]

df_results_val = pd.DataFrame()
df_results_val['y_pred'] = y_pred
df_results_val['y_true'] = y_val
df_results_val['predict_proba'] = y_pred_proba
df_results_val['average cost min'] = list(averagecostmin)
Profit_top_k(df_results_val)

3.2831259999999998

In [72]:
df_results_val

,y_pred,y_true,predict_proba,average cost min
0,1,1.0,0.910000,0.151447
1,0,0.0,0.064074,0.073770
2,1,1.0,0.836756,0.175496
3,0,0.0,0.119625,0.162873
4,0,0.0,0.052484,0.143229
...,...,...,...,...
1004,1,0.0,0.760000,0.348823
1005,0,0.0,0.210431,0.124049
1006,0,0.0,0.098195,0.190054
1007,0,0.0,0.090000,0.114843


#### Test Predictions

In [73]:
y_pred_proba_test = clf.predict_proba(X_test)
y_pred_proba_test = [sublist[1] for sublist in y_pred_proba_test]

In [74]:
df_results = pd.DataFrame()
df_results["ID"] = idtest
df_results["PRED"] = y_pred_proba_test

In [75]:
df_results.to_csv("Model_ResultsPrep2.csv", index = False)

In [76]:
print(len(y_pred))

1682


## Prep

In [180]:
X_train = encoded_train.drop(columns = "target")
y_train = encoded_train["target"]

X_val = encoded_val.drop(columns = "target")
y_val = encoded_val["target"]



X_test = encoded_test.drop(columns = "target")

y_test = encoded_test["target"].values

# Convert target labels to integers
y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)
y_test = np.array(y_test).astype(int)
y_test

array([0, 0, 0, ..., 0, 0, 0])

# Neural networks  

In [182]:
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, top_k_accuracy_score


mlp = MLPClassifier(hidden_layer_sizes=(100,), )

mlp.fit(X_train, y_train)

y_pred = mlp.predict(X_test)

accuracy = accuracy_score(y_test, y_pred) #R squred metrics
print("Neural Network Accuracy:", accuracy)

Neural Network Accuracy: 0.8525564803804994


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


## Cross Validation 

In [183]:
from sklearn.model_selection import cross_val_score, KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42) #10 fold 

#ADD profit as metrics for cross validation 

cv_results_N = cross_val_score(mlp, X_train, y_train, cv=kf, scoring= Profit_top_k) #Under the hood, cross_val_score internally calls the fit method of the estimator (model) for each fold 

print(cv_results_N)

# Calculate the mean and standard deviation of the cross-validation scores
print("Mean Accuracy:", np.mean(cv_results_N))
print("Standard Deviation of Accuracy:", np.std(cv_results_N))

/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 105, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/var/folders/n3/j9s0z6s92fd1_xkmg3xk0twh0000gn/T/ipykernel_31443/2454405933.py", line 2, in Profit_top_k
    sorted_df = df_results.sort_values(by= "predict_proba", ascending = False)
AttributeError: 'MLPClassif

[nan nan nan nan nan]
Mean Accuracy: nan
Standard Deviation of Accuracy: nan


/opt/anaconda3/lib/python3.9/site-packages/sklearn/neural_network/_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 105, in __call__
    score = scorer(estimator, *args, **kwargs)
  File "/var/folders/n3/j9s0z6s92fd1_xkmg3xk0twh0000gn/T/ipykernel_31443/2454405933.py", line 2, in Profit_top_k
    sorted_df = df_results.sort_values(by= "predict_proba", ascending = False)
AttributeError: 'MLPClassif

### to CSV

In [163]:
X_test

,Age,Dropped_Calls,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,AvePeak,AveOffPeak,AveWeekend,...,tariff_CAT 100,tariff_CAT 200,tariff_CAT 50,tariff_Play 100,tariff_Play 300,Usage_Band_High,Usage_Band_Low,Usage_Band_Med,Usage_Band_MedHigh,Usage_Band_MedLow
0,0.117647,0.2,0.196429,0.17412,0.075472,0.024478,0.027643,0.008166,0.000262,0.013685,...,0,1,0,0,0,0,0,0,1,0
1,0.617647,0.0,0.166071,0.505744,0.075472,0.085822,0.147729,0.028541,0.008096,0.036838,...,0,0,0,0,1,0,0,1,0,0
2,0.235294,0.0,0.160714,0.139979,0.09434,0.116623,0.525887,0.002506,0.008828,0.015865,...,0,1,0,0,0,1,0,0,0,0
3,0.205882,0.333333,0.225,0.328806,0.0,0.014754,0.186974,0.010196,-0.002175,0.06468,...,0,1,0,0,0,0,0,0,1,0
4,0.323529,0.0,0.130357,0.129393,0.103774,0.229439,0.086465,0.007551,0.009914,0.020227,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.294118,0.0,0.014286,0.039367,0.424528,0.515971,0.092166,0.000682,0.004807,0.011009,...,0,1,0,0,0,0,0,0,1,0
1678,0.102941,0.0,-0.001786,0.067495,0.018868,0.140253,0.059921,-0.002266,0.009441,0.06524,...,1,0,0,0,0,0,0,1,0,0
1679,0.323529,0.0,0.0375,0.255168,0.018868,0.165996,0.201105,0.006873,0.076621,0.078978,...,0,1,0,0,0,1,0,0,0,0
1680,0.132353,0.133333,0.135714,0.180316,-0.009434,0.09011,0.146778,-0.003299,0.008065,0.02601,...,1,0,0,0,0,0,0,1,0,0


In [164]:
df_results = pd.DataFrame()
df_results['ID'] = idtest

df_results['PRED'] = y_pred
df_results
df_results.to_csv("Model_results_11.csv", index = False)

# Decision Trees

In [165]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score


clf = DecisionTreeClassifier()


clf.fit(X_train, y_train)


y_pred = clf.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy:", accuracy)


Decision Tree Accuracy: 0.8442330558858502


## Cross Validation

In [166]:
kf = KFold(n_splits=10, shuffle=True, random_state=42) #10 fold 

cv_results_D = cross_val_score(clf, X_train, y_train, cv=kf)

print(cv_results_D)


# Calculate the mean and standard deviation of the cross-validation scores
print("Mean Accuracy:", np.mean(cv_results_D))
print("Standard Deviation of Accuracy:", np.std(cv_results_D))

[0.87623762 0.88366337 0.92326733 0.88366337 0.91336634 0.92555831
 0.88833747 0.87344913 0.90322581 0.88585608]
Mean Accuracy: 0.8956624818809423
Standard Deviation of Accuracy: 0.018261860385185836


# Logistic Regression 

In [171]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression()

logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", accuracy)

Logistic Regression Accuracy: 0.8733650416171225


## Cross Validation

In [172]:
kf = KFold(n_splits=10, shuffle=True, random_state=42) #10 fold 

cv_results_L = cross_val_score(logreg, X_train, y_train, cv=kf)

print(cv_results_L)

# Calculate the mean and standard deviation of the cross-validation scores
print("Mean Accuracy:", np.mean(cv_results_L))
print("Standard Deviation of Accuracy:", np.std(cv_results_L))

[0.92574257 0.92574257 0.93811881 0.91831683 0.91831683 0.93052109
 0.92307692 0.91315136 0.90818859 0.9280397 ]
Mean Accuracy: 0.9229215291256173
Standard Deviation of Accuracy: 0.008281119514263519


 # Model comparaison

In [173]:
model_results = pd.DataFrame()
model_results["Neural networks"] = [np.mean(cv_results_N),np.std(cv_results_N)]
model_results["Decision Trees"] = [np.mean(cv_results_D),np.std(cv_results_D)]
model_results["Logistic Regression"] =  [np.mean(cv_results_L),np.std(cv_results_L)]
row_names = ['Mean Accuracy', 'Standard Deviation of Accuracy']  # List of row names
model_results.index = row_names

model_results

,Neural networks,Decision Trees,Logistic Regression
Mean Accuracy,0.930112,0.895662,0.922922
Standard Deviation of Accuracy,0.006444,0.018262,0.008281


In [134]:
X_test

,Age,Dropped_Calls,OffPeak_calls_Sum,OffPeak_mins_Sum,Weekend_calls_Sum,Weekend_mins_Sum,International_mins_Sum,AvePeak,AveOffPeak,AveWeekend,...,tariff_CAT 100,tariff_CAT 200,tariff_CAT 50,tariff_Play 100,tariff_Play 300,Usage_Band_High,Usage_Band_Low,Usage_Band_Med,Usage_Band_MedHigh,Usage_Band_MedLow
0,0.117647,0.2,0.196429,0.17412,0.075472,0.024478,0.027643,0.008166,0.000262,0.013685,...,0,1,0,0,0,0,0,0,1,0
1,0.617647,0.0,0.166071,0.505744,0.075472,0.085822,0.147729,0.028541,0.008096,0.036838,...,0,0,0,0,1,0,0,1,0,0
2,0.235294,0.0,0.160714,0.139979,0.09434,0.116623,0.525887,0.002506,0.008828,0.015865,...,0,1,0,0,0,1,0,0,0,0
3,0.205882,0.333333,0.225,0.328806,0.0,0.014754,0.186974,0.010196,-0.002175,0.06468,...,0,1,0,0,0,0,0,0,1,0
4,0.323529,0.0,0.130357,0.129393,0.103774,0.229439,0.086465,0.007551,0.009914,0.020227,...,1,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,0.294118,0.0,0.014286,0.039367,0.424528,0.515971,0.092166,0.000682,0.004807,0.011009,...,0,1,0,0,0,0,0,0,1,0
1678,0.102941,0.0,-0.001786,0.067495,0.018868,0.140253,0.059921,-0.002266,0.009441,0.06524,...,1,0,0,0,0,0,0,1,0,0
1679,0.323529,0.0,0.0375,0.255168,0.018868,0.165996,0.201105,0.006873,0.076621,0.078978,...,0,1,0,0,0,1,0,0,0,0
1680,0.132353,0.133333,0.135714,0.180316,-0.009434,0.09011,0.146778,-0.003299,0.008065,0.02601,...,1,0,0,0,0,0,0,1,0,0
